In [2]:
%load_ext nb_mypy

Version 1.0.5


# Problem with TypeAliases

In [6]:
from typing import Generic, TypeVar, TypeAlias, Any, final
from abc import abstractmethod

from sagemaker.processing import Processor
from sagemaker.estimator import EstimatorBase

from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep, \
    TuningStep, ConfigurableRetryStep

# todo: decide how stringent this should be while still allowing user to add new step types
StepType = TypeVar("StepType", bound=ConfigurableRetryStep)
StepActor: TypeAlias = Processor |  EstimatorBase  # todo: add more types as needed
ConcreteStep: TypeAlias = ProcessingStep | TrainingStep | CreateModelStep | TransformStep | TuningStep


class StepFactoryInterface(Generic[StepType]):
    # @abstractmethod
    # def __init__(self, step: StepType):
    #     self.step_cls = StepType

    @property
    @abstractmethod
    # Note: This needs to return a *concrete* subclass of ConfigurableRetryStep
    def step_cls(self) -> ConcreteStep:
        ...

    def step_type(self) -> str:
        map = {
            ProcessingStep: 'processor',
        }

        return map[self.step_cls]

    @abstractmethod
    def _instantiate_step_actor(self) -> StepActor:
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        ...

    @abstractmethod
    def _construct_run_args(self) -> dict[str, Any]:  # todo: create dataclass for return types
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        ...

    def type(self):
        return StepType

    @final
    def create_step(self) -> StepType:
        step_actor: StepActor = self._instantiate_step_actor()
        run_args: dict[str, Any] = self._construct_run_args()
        # Type checker complains because you can't instantiate a ConfigurableRetryStep directly
        # (it's abstract). Ignore for now, as I don't think it's easily possible to specify we're
        # passing a concrete subclass.
        return self.step_cls(
            **{step: step_actor},
            **run_args
        )

ProcessingStepFactoryInterface = StepFactoryInterface[ProcessingStep]
psf = ProcessingStepFactoryInterface()

<cell>32: error: Invalid index type "ProcessingStep | TrainingStep | CreateModelStep | TransformStep | TuningStep" for "dict[type[ProcessingStep], str]"; expected type "type[ProcessingStep]"  [index]
<cell>54: error: "ProcessingStep" not callable  [operator]
<cell>54: error: "TrainingStep" not callable  [operator]
<cell>54: error: "CreateModelStep" not callable  [operator]
<cell>54: error: "TransformStep" not callable  [operator]
<cell>54: error: "TuningStep" not callable  [operator]
<cell>55: error: Name "step" is not defined  [name-defined]
<cell>60: error: Cannot instantiate abstract class "BaseStepFactoryInterface" with abstract attributes "_construct_run_args", "_instantiate_step_actor" and "step_cls"  [abstract]


# Trying virtual subclasses


In [13]:
from typing import Generic, TypeVar, TypeAlias, Any, final
from abc import ABC, abstractmethod

from sagemaker.processing import Processor
from sagemaker.estimator import EstimatorBase

from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep, \
    TuningStep, ConfigurableRetryStep

StepActor: TypeAlias = Processor |  EstimatorBase  # todo: add more types as needed
class ConcreteStep(ABC):
    ...

ConcreteStep.register(ProcessingStep)
ConcreteStep.register(TrainingStep)
ConcreteStep.register(CreateModelStep)
ConcreteStep.register(TransformStep)
ConcreteStep.register(TuningStep)

# todo: decide how stringent this should be while still allowing user to add new step types
StepType = TypeVar("StepType", bound=ConcreteStep)
class StepFactoryInterface(Generic[StepType]):
    @abstractmethod
    def _instantiate_step_actor(self) -> StepActor:
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        ...

    @abstractmethod
    def construct_run_args(self) -> dict[str, Any]:  # todo: create dataclass for return types
        """Note: It is consistent with LSP for implementation to return a more *specific* type."""
        # Changed: needs to include stepactor, e.g. processor=processor
        ...

    @final
    def create_step(self) -> StepType:
        step_actor: StepActor = self._instantiate_step_actor()
        run_args: dict[str, Any] = self.construct_run_args()
        # Type checker complains because you can't instantiate a ConfigurableRetryStep directly
        # (it's abstract). Ignore for now, as I don't think it's easily possible to specify we're
        # passing a concrete subclass.
        return StepType(
            **run_args
        )

ProcessingStepFactoryInterface = StepFactoryInterface[ProcessingStep]
# psf = ProcessingStepFactoryInterface()

<cell>41: error: "StepType" is a type variable and only valid in type context  [misc]
<cell>45: error: Type argument "ProcessingStep" of "BaseStepFactoryInterface" must be a subtype of "ConcreteStep"  [type-var]
<cell>45: error: Value of type variable "StepType" of "BaseStepFactoryInterface" cannot be "ProcessingStep"  [type-var]
<cell>46: error: Value of type variable "StepType" of "BaseStepFactoryInterface" cannot be "ProcessingStep"  [type-var]
<cell>46: error: Cannot instantiate abstract class "BaseStepFactoryInterface" with abstract attributes "_instantiate_step_actor" and "construct_run_args"  [abstract]


# Only create basic interface

In [16]:
from typing import Generic, TypeVar, TypeAlias, Any, final
from abc import abstractmethod

from sagemaker.processing import Processor
from sagemaker.estimator import EstimatorBase

from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep, \
    TuningStep, ConfigurableRetryStep

# todo: decide how stringent this should be while still allowing user to add new step types
StepType = TypeVar("StepType", bound=ConfigurableRetryStep)


class StepFactoryInterface(Generic[StepType]):
    @abstractmethod
    def create_step(self) -> StepType:
        ...

ProcessingStepFactoryInterface = StepFactoryInterface[ProcessingStep]


In [17]:
from sagemaker.workflow.steps import ProcessingStep


class FrameworkProcessorFactory(ProcessingStepFactoryInterface):
    def create_step(self) -> ProcessingStep:
        ...

<cell>5: error: Missing return statement  [empty-body]
